# Classification

In [ ]:
# Data Handling and Utilities
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
import joblib

# Audio Processing
import librosa
import librosa.display

# Image Processing
from skimage.transform import resize

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Machine Learning
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Evaluation Metrics
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import load_model


In [ ]:
df_train = pd.read_csv("numeric_train_data.csv")
df_test = pd.read_csv("numeric_test_data.csv")
df_train.head(5)
df_test.head(5)

,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,...,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,label
0,0.715236,0.693741,0.673291,0.675155,0.718251,0.703219,0.706388,0.757179,0.673783,0.617892,...,0.438356,0.592341,0.422709,0.600324,0.491890,0.654020,0.516087,0.745383,0.426049,0
1,0.638202,0.539125,0.413576,0.500710,0.633080,0.634422,0.585670,0.579234,0.597464,0.582047,...,0.351815,0.528008,0.416186,0.631978,0.522208,0.740917,0.466066,0.713554,0.401167,0
2,0.300166,0.289997,0.279196,0.318552,0.343168,0.292658,0.251033,0.396900,0.611866,0.711495,...,0.113269,0.463730,0.372439,0.567525,0.539345,0.616263,0.191980,0.342027,0.198181,1
3,0.697828,0.647420,0.581630,0.592989,0.683064,0.710182,0.698529,0.667966,0.691158,0.685162,...,0.419216,0.668173,0.503341,0.626000,0.538417,0.697739,0.478217,0.811338,0.518223,0
4,0.370128,0.508962,0.492916,0.391020,0.363103,0.367662,0.399747,0.469762,0.498174,0.399316,...,0.409716,0.630644,0.338582,0.540950,0.436101,0.765488,0.450831,0.634361,0.329778,1


In [ ]:
df_train = df_train
df_test  = df_test

X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]

X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]


In [4]:
data_sample = df_test.sample(1000, random_state=42)
X_sample = data_sample.iloc[:, :-1]
y_sample = data_sample.iloc[:, -1]

### Random Forest

In [7]:
feature_rf = [
    "chroma_2", "chroma_3", "chroma_4", "chroma_6", "chroma_7",
    "chroma_8", "chroma_9", "chroma_10", "chroma_11", "chroma_12",
    "rms_mean", "spectral_centroid_mean", "spectral_bandwidth_mean",
    "spectral_rolloff_mean", "zcr_mean",
    "mfcc_1", "mfcc_2", "mfcc_3", "mfcc_4", "mfcc_5", "mfcc_6",
    "mfcc_7", "mfcc_8", "mfcc_9", "mfcc_10", "mfcc_11", "mfcc_12",
    "mfcc_13", "mfcc_14", "mfcc_15", "mfcc_16", "mfcc_17", "mfcc_18",
    "mfcc_19", "mfcc_20"
]

X_train_rf = X_train[feature_rf]
X_test_rf = X_test[feature_rf]

X_sample_rf = X_sample[feature_rf]

In [ ]:


rf_model = RandomForestClassifier(
    n_estimators=278,
    max_depth=18,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features=0.5,
    class_weight=None,
    random_state=42,
    n_jobs=1
)


# Latih model pada data training
rf_model.fit(X_train_rf, y_train)

# ===== Evaluasi pada Data Training =====
y_pred_train_rf = rf_model.predict(X_train_rf)
train_acc_rf = accuracy_score(y_train, y_pred_train_rf)
print(f"Train Accuracy (RF): {train_acc_rf * 100:.2f}%")

# ===== Evaluasi pada Data Test =====
y_pred_test_rf = rf_model.predict(X_test_rf)
test_acc_rf = accuracy_score(y_test, y_pred_test_rf)
print(f"Test Accuracy (RF): {test_acc_rf * 100:.2f}%")



Train Accuracy (RF): 98.87%
Test Accuracy (RF): 94.50%


In [ ]:
import joblib

# Load the saved model
rf_model = joblib.load('29mei_random_forest_model.pkl')



d:\Anacondas\envs\tf128\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Anacondas\envs\tf128\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:


# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_rf = []

# Loop untuk menghitung waktu prediksi satu per satu untuk setiap sampel
for i in range(1, len(X_sample_rf) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = rf_model.predict(X_sample_rf.iloc[i-1:i, :])
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # hanya satu sampel

    prediction_times_rf.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_rf = pd.DataFrame(prediction_times_rf)

# Simpan ke CSV
df_times_rf.to_csv("rf_prediction_per_sample.csv", index=False)

print("\n✅ Waktu prediksi per sampel berhasil disimpan ke 'rf_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_rf['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁

In [11]:
import gc
del rf_model
gc.collect()


544

### SVM

In [10]:
feature_svm = [
    "chroma_1", "chroma_2", "chroma_4", "chroma_9", "chroma_10", "chroma_11",
    "rms_mean", "spectral_centroid_mean", "spectral_bandwidth_mean",
    "spectral_rolloff_mean", "zcr_mean",
    "mfcc_1", "mfcc_2", "mfcc_3", "mfcc_4", "mfcc_5", "mfcc_6", "mfcc_7",
    "mfcc_9", "mfcc_11", "mfcc_12", "mfcc_13", "mfcc_14", "mfcc_15", "mfcc_18"
]

X_train_svm = X_train[feature_svm]
X_test_svm = X_test[feature_svm]
X_sample_svm = X_sample[feature_svm]

In [ ]:
# Inisialisasi model SVM dengan hyperparameter terbaik
svm_model = SVC(
    kernel='rbf',
    C=98.03363720445789,
    gamma=0.9926573491055003,
    random_state=42
)

# Latih model pada data training
svm_model.fit(X_train_svm, y_train)

# ===== Evaluasi pada Data Training =====
y_pred_train_svm = svm_model.predict(X_train_svm)
train_acc_svm = accuracy_score(y_train, y_pred_train_svm)
print(f"Train Accuracy (SVM): {train_acc_svm * 100:.2f}%")

# ===== Evaluasi pada Data Test =====
y_pred_test_svm = svm_model.predict(X_test_svm)
test_acc_svm = accuracy_score(y_test, y_pred_test_svm)
print(f"Test Accuracy (SVM): {test_acc_svm * 100:.2f}%")



In [12]:
import joblib

# Load the saved model
svm_model = joblib.load('29mei_SVM_model.pkl')

# Now you can use rf_model for prediction or other tasks
# Example:
# predictions = rf_model.predict(X_test)


d:\Anacondas\envs\tf128\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:


# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_svm = []

# Loop untuk menghitung waktu prediksi per 1 sampel
for i in range(1, len(X_sample_svm) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = svm_model.predict(X_sample_svm.iloc[i-1:i, :])
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # satu sampel, jadi waktu per sampel = total waktu

    prediction_times_svm.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_svm = pd.DataFrame(prediction_times_svm)

# Simpan ke CSV
df_times_svm.to_csv("svm_prediction_per_sample.csv", index=False)

print("✅ Waktu prediksi per sampel berhasil disimpan ke 'svm_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_svm['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁

### Model CNN

In [ ]:
data_val_cnn = np.load("cnn_test_data.npz")
X = data_val_cnn["data"]

# number of samples in X
n_samples = X.shape[0]

# set random seed for reproducibility
rng = np.random.default_rng(42)

# sample 1000 random indices without replacement
indices = rng.choice(n_samples, size=1000, replace=False)

# select those rows
X_sample_cnn = X[indices]


In [ ]:


# Ganti 'model_name.h5' dengan path file model .h5 Anda
model_cnn = load_model('30mei_model_cnn.h5')



In [ ]:


# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_cnn = []

# Loop untuk menghitung waktu prediksi satu per satu untuk setiap sampel
for i in range(1, len(X_sample_cnn) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = model_cnn.predict(X_sample_cnn[i-1:i], verbose = 0)  # pastikan input shape sesuai (misalnya [1, height, width, channels])
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # hanya satu sampel

    prediction_times_cnn.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_cnn = pd.DataFrame(prediction_times_cnn)

# Simpan ke CSV
df_times_cnn.to_csv("cnn_prediction_per_sample.csv", index=False)

print("\n✅ Waktu prediksi per sampel berhasil disimpan ke 'cnn_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_cnn['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁

In [ ]:
import gc
del model_cnn
gc.collect()

9838

Model CNN-LSTM

In [ ]:
data_val_cnn_lstm = np.load("cnn_lstm_test_data.npz")
X = data_val_cnn_lstm["data"]

n_samples = X.shape[0]

rng = np.random.default_rng(42)  # random generator with seed for reproducibility
indices = rng.choice(n_samples, size=1000, replace=False)

X_sample_cnn_lstm = X[indices]


In [ ]:


# Ganti 'model_name.h5' dengan path file model .h5 Anda
model_cnn_lstm = load_model('30mei_model_cnn_lstm_3.h5')

In [4]:
# Inisialisasi list untuk menyimpan hasil waktu prediksi per sampel
prediction_times_cnn_lstm = []

# Loop untuk menghitung waktu prediksi satu per satu untuk setiap sampel
for i in range(1, len(X_sample_cnn_lstm) + 1):
    print(f"🔁 Iterasi ke-{i}")

    start_time = time.perf_counter()
    _ = model_cnn_lstm.predict(X_sample_cnn_lstm[i-1:i], verbose=0)  # asumsi input batch shape sudah sesuai
    elapsed_time = time.perf_counter() - start_time
    time_per_sample = elapsed_time  # hanya satu sampel

    prediction_times_cnn_lstm.append({
        "Iteration": i,
        "Total_Prediction_Time_sec": elapsed_time,
        "Time_per_Sample_sec": time_per_sample
    })

# Simpan hasil ke DataFrame
df_times_cnn_lstm = pd.DataFrame(prediction_times_cnn_lstm)

# Simpan ke CSV
df_times_cnn_lstm.to_csv("cnn_lstm_prediction_per_sample.csv", index=False)

print("\n✅ Waktu prediksi per sampel berhasil disimpan ke 'cnn_lstm_prediction_per_sample.csv'")
print("Rata-rata waktu prediksi per sampel:", df_times_cnn_lstm['Time_per_Sample_sec'].mean(), "detik")


🔁 Iterasi ke-1
🔁 Iterasi ke-2
🔁 Iterasi ke-3
🔁 Iterasi ke-4
🔁 Iterasi ke-5
🔁 Iterasi ke-6
🔁 Iterasi ke-7
🔁 Iterasi ke-8
🔁 Iterasi ke-9
🔁 Iterasi ke-10
🔁 Iterasi ke-11
🔁 Iterasi ke-12
🔁 Iterasi ke-13
🔁 Iterasi ke-14
🔁 Iterasi ke-15
🔁 Iterasi ke-16
🔁 Iterasi ke-17
🔁 Iterasi ke-18
🔁 Iterasi ke-19
🔁 Iterasi ke-20
🔁 Iterasi ke-21
🔁 Iterasi ke-22
🔁 Iterasi ke-23
🔁 Iterasi ke-24
🔁 Iterasi ke-25
🔁 Iterasi ke-26
🔁 Iterasi ke-27
🔁 Iterasi ke-28
🔁 Iterasi ke-29
🔁 Iterasi ke-30
🔁 Iterasi ke-31
🔁 Iterasi ke-32
🔁 Iterasi ke-33
🔁 Iterasi ke-34
🔁 Iterasi ke-35
🔁 Iterasi ke-36
🔁 Iterasi ke-37
🔁 Iterasi ke-38
🔁 Iterasi ke-39
🔁 Iterasi ke-40
🔁 Iterasi ke-41
🔁 Iterasi ke-42
🔁 Iterasi ke-43
🔁 Iterasi ke-44
🔁 Iterasi ke-45
🔁 Iterasi ke-46
🔁 Iterasi ke-47
🔁 Iterasi ke-48
🔁 Iterasi ke-49
🔁 Iterasi ke-50
🔁 Iterasi ke-51
🔁 Iterasi ke-52
🔁 Iterasi ke-53
🔁 Iterasi ke-54
🔁 Iterasi ke-55
🔁 Iterasi ke-56
🔁 Iterasi ke-57
🔁 Iterasi ke-58
🔁 Iterasi ke-59
🔁 Iterasi ke-60
🔁 Iterasi ke-61
🔁 Iterasi ke-62
🔁 Iterasi ke-63
🔁